#Virtual Memory:

Virtual memory management involves translating virtual addresses used by a program into physical addresses in RAM. This includes handling page faults and swapping pages between RAM and disk when necessary.

In [ ]:
class PageTable:
    def __init__(self, num_entries):
        self.num_entries = num_entries
        self.table = {}

    def map_virtual_to_physical(self, virtual_address):
        page_number = virtual_address // PAGE_SIZE
        if page_number in self.table:
            return self.table[page_number]
        else:
            raise PageFaultError(f"Page {page_number} is not in physical memory")

    def update_page_table_entry(self, page_number, frame_number):
        self.table[page_number] = frame_number


class MemoryManager:
    def __init__(self, memory_size, page_size):
        self.memory_size = memory_size
        self.page_size = page_size
        self.num_frames = memory_size // page_size
        self.frames = [None] * self.num_frames
        self.page_table = PageTable(num_entries=self.num_frames)
        self.free_frame_list = list(range(self.num_frames))

    def allocate_frame(self):
        if len(self.free_frame_list) > 0:
            return self.free_frame_list.pop(0)
        else:
            raise MemoryFullError("No free frames available in physical memory")

    def load_page_into_memory(self, page_number):
        frame_number = self.allocate_frame()
        self.frames[frame_number] = f"Page {page_number}"
        self.page_table.update_page_table_entry(page_number, frame_number)

    def access_virtual_memory(self, virtual_address):
        try:
            frame_number = self.page_table.map_virtual_to_physical(virtual_address)
            print(f"Accessing physical memory: {self.frames[frame_number]}")
        except PageFaultError as e:
            print(f"Page fault occurred: {e}")
            # Load page from disk into memory
            page_number = virtual_address // self.page_size
            self.load_page_into_memory(page_number)
            self.access_virtual_memory(virtual_address)  # Retry access after page is loaded


# Constants
PAGE_SIZE = 4096  # Page size in bytes


class PageFaultError(Exception):
    pass


class MemoryFullError(Exception):
    pass


# Usage example:
memory_manager = MemoryManager(memory_size=16384, page_size=PAGE_SIZE)

# Access virtual memory addresses
addresses = [8192, 4096, 12288, 16384]

for address in addresses:
    memory_manager.access_virtual_memory(address)

Page fault occurred: Page 2 is not in physical memory
Accessing physical memory: Page 2
Page fault occurred: Page 1 is not in physical memory
Accessing physical memory: Page 1
Page fault occurred: Page 3 is not in physical memory
Accessing physical memory: Page 3
Page fault occurred: Page 4 is not in physical memory
Accessing physical memory: Page 4
